# 🧠 Understanding Function Calling with OpenAI API (Function Tool Calling)

Welcome to this tutorial on **Function Calling** with the OpenAI API! This powerful feature allows you to connect large language models to external tools by enabling them to call specific functions you define. By the end of this lesson, you'll understand how to build more dynamic and capable applications like AI agents, chatbots, and smart automation systems.

### What is Function Calling?

Function calling lets the model intelligently decide when to call a function you've provided based on the user's prompt. For example, if a user asks, "What's the weather in London?", you can have a `get_weather` function that the model can choose to call. The process works like this:

1.  **You define a function** and provide its description to the model.
2.  The model **analyzes the user's prompt** and determines if it should call your function.
3.  If it decides to, the model **returns a JSON object** with the function name and the arguments to use.
4.  You **execute the function** with those arguments and send the result back to the model.
5.  The model uses this information to **generate a final response** to the user.

Let's dive in!

## 🛠️ Setup: Installing Libraries

First, we need to install the necessary Python libraries. We'll be using `openai` for interacting with the API and `python-dotenv` for managing our API key securely.

In [17]:
import IPython
import sys

def clean_notebook():
    IPython.display.clear_output(wait=True)
    print("Notebook cleaned.")

# Install the required packages
!pip install openai python-dotenv

# Clean up the notebook output
clean_notebook()

Notebook cleaned.


## 🔑 Loading Environment Variables

To keep our API key secure, we'll store it in a `.env` file and load it into our environment. Make sure you have a file named `.env` in the same directory with your `OPENAI_API_KEY`.

In [18]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

## 🤖 Initializing the OpenAI Client

Now, let's create an instance of the OpenAI client. We'll use this client to make requests to the API. We are specifying the model we want to use, in this case, `gpt-4.1`.

In [19]:
import os
from openai import OpenAI

# Initialize the OpenAI client with the API key
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model_name = "gpt-4.1"

## ☀️ Example: A Mock Weather Function

Here, we'll define a simple function called `get_weather`. In a real-world application, this function would call a weather API. For this example, it will just return a fixed string. This helps us focus on the function calling mechanism itself.

In [20]:
import json

# Define the mock function
def get_weather(location):
    """A mock function to get the current temperature."""
    return f"The current temperature in {location} is 72°F."

# Define the 'tool' for the model to use
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current temperature for a given location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and country, e.g., Bogotá, Colombia"
                    }
                },
                "required": ["location"]
            }
        }
    }
]

## 🗣️ Making the First API Call

Now, let's ask a question that should trigger our `get_weather` function. We'll send the user's query to the model and tell it about the tools it can use.

In [ ]:
# The user's query
messages = [{"role": "user", "content": "What is the weather like in Paris today?"}]

# Make the first API call
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    tool_choice="auto"  # Let the model decide whether to call a function
)

# Get the model's response message
response_message = response.choices[0].message
response_message

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_gDwk6kVfd9HxqdAqvtDPnVVW', function=Function(arguments='{"location":"Paris, France"}', name='get_weather'), type='function')])

## 🤔 Analyzing the Model's Response

Let's inspect the response. If the model decided to call a function, the `content` of the message will be `None`, and it will include `tool_calls` with the function name and arguments.

In [22]:
# Display the initial response from the model
print("Model's response:", response_message.content)

# Check if the model wants to call our function
if response_message.tool_calls:
    tool_call = response_message.tool_calls[0]
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)
    print(f"Function call: {function_name} with arguments {function_args}")

Model's response: None
Function call: get_weather with arguments {'location': 'Paris, France'}


## 🧪 Testing with Multiple Questions

To see how well the model distinguishes between different types of questions, let's test it with a list of queries. Some of these should trigger our `get_weather` function, while others should be answered directly.

In [23]:
import json
from openai import OpenAI

client = OpenAI()  # Assume API key is set

# Our trusty mock function
def get_weather(location):
    return f"The current temperature in {location} is 72°F."

# The tool definition remains the same
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current temperature for a given location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and country, e.g., Bogotá, Colombia"
                    }
                },
                "required": ["location"]
            }
        }
    }
]

# A list of test questions in Thai
test_questions = [
    "วันนี้สภาพอากาศในปารีสเป็นอย่างไร?",
    "อุณหภูมิในนิวยอร์กซิตี้เป็นเท่าไหร่?",
    "แมวชอบกินอาหารอะไร?",
    "สภาพอากาศในโตเกียว ญี่ปุ่น เป็นอย่างไร?",
    "บอกอุณหภูมิปัจจุบันในลอนดอนให้หน่อย",
    "รายงานสภาพอากาศสำหรับโบโกตา โคลอมเบีย",
    "ตอนนี้ในซิดนีย์ร้อนไหม?",
    "สภาพอากาศปัจจุบันในเบอร์ลิน",
    "สองบวกสองเท่ากับเท่าไหร่?",
    "เมืองหลวงของฝรั่งเศสคืออะไร?",
    "เล่าเรื่องตลกให้ฟังหน่อย"
]

# Loop through each question and see how the model responds
for query in test_questions:
    messages = [{"role": "user", "content": query}]
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    response_message = response.choices[0].message
    
    print(f"Test Query: {query}")
    print("Initial Model's response content:", response_message.content)
    
    if response_message.tool_calls:
        tool_call = response_message.tool_calls[0]
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        print(f"Function call: {function_name} with arguments {function_args}")
        
        if function_name == "get_weather":
            function_response = get_weather(**function_args)
            print("Function response:", function_response)
            
    print("-" * 50)
    print()  # Add a newline for readability

Test Query: วันนี้สภาพอากาศในปารีสเป็นอย่างไร?
Initial Model's response content: None
Function call: get_weather with arguments {'location': 'Paris, France'}
Function response: The current temperature in Paris, France is 72°F.
--------------------------------------------------

Test Query: อุณหภูมิในนิวยอร์กซิตี้เป็นเท่าไหร่?
Initial Model's response content: None
Function call: get_weather with arguments {'location': 'New York City, USA'}
Function response: The current temperature in New York City, USA is 72°F.
--------------------------------------------------

Test Query: แมวชอบกินอาหารอะไร?
Initial Model's response content: แมวชอบกินอาหารที่มีโปรตีนสูง เช่น เนื้อปลา เนื้อสัตว์ปีก เนื้อวัว และผลิตภัณฑ์จากนม นอกจากนี้ อาหารแมวสำเร็จรูปที่มีสารอาหารครบถ้วนก็เป็นที่นิยมเช่นกัน การเลือกอาหารที่เหมาะสมกับอายุกับสุขภาพของแมวก็เป็นสิ่งสำคัญ ถ้ามีปัญหาสุขภาพควรปรึกษาสัตวแพทย์เพื่อเลือกอาหารที่เหมาะสม
--------------------------------------------------

Test Query: สภาพอากาศในโตเกียว ญี่ปุ่น 

# Multiple Function Calling

In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [3]:
import os
import json
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model_name = "gpt-4.1"    


In [30]:


# Define mock functions
def get_weather(location):
    """Mock weather function - in real scenario, call weather API"""
    weather_data = {
        "Paris, France": "Sunny, 22°C",
        "Bangkok, Thailand": "Cloudy, 28°C", 
        "New York, USA": "Rainy, 18°C",
        "Tokyo, Japan": "Partly cloudy, 25°C"
    }
    return weather_data.get(location, f"Weather data not available for {location}")

def get_time(location):
    """Mock time function - in real scenario, call timezone API"""
    time_data = {
        "Paris, France": "14:30 CET",
        "Bangkok, Thailand": "20:30 ICT",
        "New York, USA": "08:30 EST", 
        "Tokyo, Japan": "22:30 JST"
    }
    return time_data.get(location, f"Time data not available for {location}")

def get_exchange_rate(from_currency, to_currency):
    """Mock exchange rate function"""
    rates = {
        ("USD", "EUR"): 0.85,
        ("USD", "THB"): 33.50,
        ("EUR", "USD"): 1.18,
        ("EUR", "THB"): 39.40
    }
    rate = rates.get((from_currency, to_currency))
    if rate:
        return f"1 {from_currency} = {rate} {to_currency}"
    else:
        return f"Exchange rate not available for {from_currency} to {to_currency}"

# Define tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather for a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City and country e.g. Paris, France"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function", 
        "function": {
            "name": "get_time",
            "description": "Get current time for a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City and country e.g. Paris, France"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_exchange_rate", 
            "description": "Get exchange rate between two currencies",
            "parameters": {
                "type": "object",
                "properties": {
                    "from_currency": {
                        "type": "string",
                        "description": "Source currency code e.g. USD"
                    },
                    "to_currency": {
                        "type": "string", 
                        "description": "Target currency code e.g. EUR"
                    }
                },
                "required": ["from_currency", "to_currency"]
            }
        }
    }
]

# Function mapping
available_functions = {
    "get_weather": get_weather,
    "get_time": get_time, 
    "get_exchange_rate": get_exchange_rate
}




In [31]:

# User query that might trigger multiple function calls
messages = [
    {
        "role": "user", 
        "content": "I'm planning a trip to Paris and Bangkok. Can you tell me the weather and current time in both cities? Also, what's the exchange rate from USD to EUR and USD to THB?"
    }
]


messages

[{'role': 'user',
  'content': "I'm planning a trip to Paris and Bangkok. Can you tell me the weather and current time in both cities? Also, what's the exchange rate from USD to EUR and USD to THB?"}]

In [32]:

# First API call
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

response_message = response.choices[0].message
tool_calls = response_message.tool_calls

print("=== Initial Response ===")
print(f"Content: {response_message.content}")
print(f"Tool calls: {len(tool_calls) if tool_calls else 0}")
print()

# Add the assistant's response to messages (including tool calls)
messages.append({
    "role": "assistant",
    "content": response_message.content,
    "tool_calls": tool_calls
})

# Process tool calls if they exist
if tool_calls:
    print("=== Processing Tool Calls ===")
    
    for tool_call in tool_calls:
        print(f"Tool call: {tool_call.function.name} with arguments {tool_call.function.arguments}")
        
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        
        if function_name in available_functions:
            # Execute the function
            function_response = available_functions[function_name](**function_args)
            print(f"Function response for {function_name}: {function_response}")
            
            # Add the function response to messages
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(function_response)
            })
        else:
            print(f"Function {function_name} not found.")
            
            # Add error response
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": f"Error: Function {function_name} not available"
            })
        
        print("-" * 50)


=== Initial Response ===
Content: None
Tool calls: 6

=== Processing Tool Calls ===
Tool call: get_weather with arguments {"location": "Paris, France"}
Function response for get_weather: Sunny, 22°C
--------------------------------------------------
Tool call: get_weather with arguments {"location": "Bangkok, Thailand"}
Function response for get_weather: Cloudy, 28°C
--------------------------------------------------
Tool call: get_time with arguments {"location": "Paris, France"}
Function response for get_time: 14:30 CET
--------------------------------------------------
Tool call: get_time with arguments {"location": "Bangkok, Thailand"}
Function response for get_time: 20:30 ICT
--------------------------------------------------
Tool call: get_exchange_rate with arguments {"from_currency": "USD", "to_currency": "EUR"}
Function response for get_exchange_rate: 1 USD = 0.85 EUR
--------------------------------------------------
Tool call: get_exchange_rate with arguments {"from_currency

In [33]:
messages

[{'role': 'user',
  'content': "I'm planning a trip to Paris and Bangkok. Can you tell me the weather and current time in both cities? Also, what's the exchange rate from USD to EUR and USD to THB?"},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [ChatCompletionMessageToolCall(id='call_1snwhGUC1AOh1oscwROnEEc5', function=Function(arguments='{"location": "Paris, France"}', name='get_weather'), type='function'),
   ChatCompletionMessageToolCall(id='call_zBWyZVcI2OOtgOzqxHcpDEiU', function=Function(arguments='{"location": "Bangkok, Thailand"}', name='get_weather'), type='function'),
   ChatCompletionMessageToolCall(id='call_psxNAvMkHkIRATGyPhMQA3hN', function=Function(arguments='{"location": "Paris, France"}', name='get_time'), type='function'),
   ChatCompletionMessageToolCall(id='call_ZnJtGHYhHRq4XhO3rwqmcAqG', function=Function(arguments='{"location": "Bangkok, Thailand"}', name='get_time'), type='function'),
   ChatCompletionMessageToolCall(id='call_1I4S43SbUTeVfc1hpcb22SO

In [34]:

# Make a second API call to get the final response with all tool results
final_response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages
)

print("=== Final Response ===")
print(final_response.choices[0].message.content)


=== Final Response ===
Here's the information for your trip:

### Current Weather
- **Paris, France:** Sunny, 22°C
- **Bangkok, Thailand:** Cloudy, 28°C

### Current Time
- **Paris, France:** 14:30 CET
- **Bangkok, Thailand:** 20:30 ICT

### Exchange Rates
- **USD to EUR:** 1 USD = 0.85 EUR
- **USD to THB:** 1 USD = 33.5 THB

Have a great trip planning! If you have any more questions or need further assistance, feel free to ask.
